# 1　データ整形：モデル前データまで

## 準備

In [1]:
from pathlib import Path
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
from xml.etree.ElementTree import fromstring, ElementTree
from Evtx.Evtx import Evtx
import csv

In [2]:
%load_ext autoreload
%autoreload 2
import preprocess
import main

/home/siwamura/My_lad/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
INTERIM_DIR = Path('../data/interim')
PROCESSED_DIR = Path('../data/processed')
RAW_DIR = Path('../data/raw')
NO_MEANING_DIR = Path('../data/no_meaning')

## 1-1 AtomicRT（T1105）

### processed 作成（使用特徴量：EventId, deltaT）

In [ ]:
input_dir = INTERIM_DIR/project_name
output_dir = PROCESSED_DIR/project_name

In [ ]:
df = pd.read_csv(input_dir / f'{file_name}_prepared_structured.csv')

# 諸操作
df["Label"] = df["Label"].apply(lambda x: int(x != "-"))
df["datetime"] = pd.to_datetime(df["Timestamp"], format="%Y-%m-%d %H:%M:%S.%f %z")
df["timestamp"] = df["datetime"].view("int64") // 10**9  
df["deltaT"] = df["datetime"].diff().dt.total_seconds().fillna(0)

/home/siwamura/lad2/src_/ipykernel_1245661/1067428620.py:6: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  df["timestamp"] = df["datetime"].view("int64") // 10**9


In [ ]:
def sliding_window(raw_data, para):
    """
    split logs into sliding windows/session
    :param raw_data: dataframe columns=[timestamp, label, eventid, time duration]
    :param para:{window_size: seconds, step_size: seconds}
    :return: dataframe columns=[eventids, time durations, label]
    """
    log_size = raw_data.shape[0]
    label_data, time_data = raw_data.iloc[:, 1], raw_data.iloc[:, 0]
    logkey_data, deltaT_data = raw_data.iloc[:, 2], raw_data.iloc[:, 3]
    new_data = []
    start_end_index_pair = set()

    start_time = time_data[0]
    end_time = start_time + para["window_size"]
    start_index = 0
    end_index = 0

    # get the first start, end index, end time
    for cur_time in time_data:
        if cur_time < end_time:
            end_index += 1
        else:
            break

    start_end_index_pair.add(tuple([start_index, end_index]))

    # move the start and end index until next sliding window
    num_session = 1
    while end_index < log_size:
        start_time = start_time + para["step_size"]
        end_time = start_time + para["window_size"]
        for i in range(start_index, log_size):
            if time_data[i] < start_time:
                i += 1
            else:
                break
        for j in range(end_index, log_size):
            if time_data[j] < end_time:
                j += 1
            else:
                break
        start_index = i
        end_index = j

        # when start_index == end_index, there is no value in the window
        if start_index != end_index:
            start_end_index_pair.add(tuple([start_index, end_index]))

        num_session += 1
        if num_session % 1000 == 0:
            print("process {} time window".format(num_session), end="\r")

    for start_index, end_index in start_end_index_pair:
        dt = deltaT_data[start_index:end_index].values
        dt[0] = 0
        new_data.append(
            [
                time_data[start_index:end_index].values,
                max(label_data[start_index:end_index]),
                logkey_data[start_index:end_index].values,
                dt,
            ]
        )

    assert len(start_end_index_pair) == len(new_data)
    print(
        "there are %d instances (sliding windows) in this dataset\n"
        % len(start_end_index_pair)
    )
    return pd.DataFrame(new_data, columns=raw_data.columns)

# dfをdeeplog_fileに変換
def deeplog_file_generator(filename, df, features):
    with open(filename, "w") as f:
        for _, row in df.iterrows():
            for val in zip(*row[features]):
                f.write(",".join([str(v) for v in val]) + " ")
            f.write("\n")

In [ ]:
window_size = 5
step_size = 1

# sampling with sliding window
deeplog_df = sliding_window(
    df[["timestamp", "Label", "EventId", "deltaT"]],
    para={"window_size": int(window_size) * 60, "step_size": int(step_size) * 60},
)

there are 820 instances (sliding windows) in this dataset



In [ ]:
# normalとabnormalを切り分け
df_normal = deeplog_df[deeplog_df["Label"] == 0]
df_abnormal = deeplog_df[deeplog_df["Label"] == 1]

df_normal = df_normal.sample(frac=1, random_state=12).reset_index(
    drop=True
)  # shuffle
normal_len = len(df_normal)

In [ ]:
df_abnormal

,timestamp,Label,EventId,deltaT


##### train_ratio = 40%, 60%, 80%

In [ ]:
# 案１（ファイル名を分けない）
train_ratio_list = [0.4, 0.6, 0.8]
for train_ratio in train_ratio_list:

    train_len = int(normal_len * train_ratio)
    save_dir = output_dir/f'ratio_{train_ratio}'

    os.makedirs(save_dir, exist_ok=True)

    # train
    train = df_normal[:train_len]
    deeplog_file_generator(
        filename = str(save_dir) + '/train',
        df = train,
        features = ["EventId", "deltaT"],
    )
    print("training size {}".format(train_len))

    # test(normal)
    test_normal = df_normal[train_len:]
    deeplog_file_generator(
        filename = str(save_dir) + '/test_normal',
        df = test_normal,
        features = ["EventId", "deltaT"],
    )
    print("test normal size {}".format(normal_len - train_len))

    # abnormal
    
    # 必要なら EventId のマッピングを復活
    # df_abnormal["EventId"] = df_abnormal["EventId"].progress_apply(
    #     lambda e: event_index_map[e] if event_index_map.get(e) else UNK
    # )

    deeplog_file_generator(
        filename = str(save_dir) + '/test_abnormal',
        df = df_abnormal,
        features = ["EventId", "deltaT"], 
    )
    print("test abnormal size {}".format(len(df_abnormal)))

training size 328
test normal size 492
test abnormal size 0
training size 492
test normal size 328
test abnormal size 0
training size 656
test normal size 164
test abnormal size 0


In [ ]:
# 案２（それぞれファイル名を分ける）

# train
train = df_normal[:train_len]
deeplog_file_generator(
    filename = str(save_dir) + f'/train_{train_ratio}',
    df = train,
    features = ["EventId", "deltaT"],
)
print("training size {}".format(train_len))

# test
test_normal = df_normal[train_len:]
deeplog_file_generator(
    filename = str(save_dir) + f'/test_{(1.0 - train_ratio):.1f}',
    df = test_normal,
    features = ["EventId", "deltaT"],
)

## 1-1 T1105
- interim下にあるsecurity.csvは10日間を抽出したものであることに注意（コードに反映していない）

### 準備

In [5]:
parent_dir = "ScenarioData"
project_name = 'T1105'
ver_number = "2"

### evtx → csv

In [8]:
# test
input_dir = RAW_DIR/parent_dir/project_name
output_dir = INTERIM_DIR/project_name

preprocess.evtx_to_csv_without_eventdata_columns_samplingver(
    evtx_filepath = input_dir/"20240927120753_7EA74D56-6663-313B-2CC1-A7843FCD1AE6/Evtx/Security.evtx",
    output_dir = output_dir,
    output_filename = "security2_test",
)

Writing to CSV: 100%|██████████| 50/50 [00:00<00:00, 71453.22it/s]


In [9]:
# 実行
preprocess.evtx_to_csv_without_eventdata_columns(
    evtx_filepath = input_dir/"20240927120753_7EA74D56-6663-313B-2CC1-A7843FCD1AE6/Evtx/Security.evtx",
    output_dir = output_dir,
    output_filename = "security2",
)

Writing to CSV: 100%|██████████| 1027696/1027696 [00:13<00:00, 74554.24it/s]


### アノテーション

In [40]:
project_name = 'T1105'
ver_number = "2"
ano = pd.read_excel(RAW_DIR/parent_dir/project_name/"取りまとめ後参考"/"AtomicRedTeam_Analyze.xlsx", sheet_name="解析結果")
df = pd.DataFrame()
df["Channel"] = ano["Unnamed: 9"].iloc[1:].values
df["EventRecordID"] = ano["Unnamed: 14"].iloc[1:].values

labeled = preprocess.anotate_csv(
    csv_filepath = INTERIM_DIR/project_name/"security2.csv",
    ano_df = df,
    output_dir = INTERIM_DIR/project_name,
)

#### 異常期間の同定
- **期間**：9/26 17:57 ~ 9/26 18:55 (73件)

In [15]:
labeled[labeled["Label"] == "anomaly"]

,Level,EventID,EventRecordID,TimeCreated_SystemTime,Channel,Task,Provider_Name,Correlation_RelatedActivityID,Execution_ProcessID,Execution_ThreadID,Security_UserID,Keywords,Content,Correlation_ActivityID,Provider_Guid,Computer,Opcode,Version,Label,date
170778,0,4688,4718654,2024-09-26 17:57:13.401667+00:00,Security,13312,Microsoft-Windows-Security-Auditing,NaN,4,300,NaN,0x8020000000000000,SubjectUserSid=S-1-5-21-143320146-2996204461-2...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},SWAttckd.swtestnet.com,0,2,anomaly,2024-09-26
170779,0,4688,4718655,2024-09-26 17:57:13.409899+00:00,Security,13312,Microsoft-Windows-Security-Auditing,NaN,4,32,NaN,0x8020000000000000,SubjectUserSid=S-1-5-21-143320146-2996204461-2...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},SWAttckd.swtestnet.com,0,2,anomaly,2024-09-26
170780,0,4688,4718656,2024-09-26 17:57:13.442478+00:00,Security,13312,Microsoft-Windows-Security-Auditing,NaN,4,32,NaN,0x8020000000000000,SubjectUserSid=S-1-5-21-143320146-2996204461-2...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},SWAttckd.swtestnet.com,0,2,anomaly,2024-09-26
170834,0,4688,4718710,2024-09-26 17:59:47.880032+00:00,Security,13312,Microsoft-Windows-Security-Auditing,NaN,4,8064,NaN,0x8020000000000000,SubjectUserSid=S-1-5-21-143320146-2996204461-2...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},SWAttckd.swtestnet.com,0,2,anomaly,2024-09-26
170835,0,4688,4718711,2024-09-26 17:59:47.883730+00:00,Security,13312,Microsoft-Windows-Security-Auditing,NaN,4,8744,NaN,0x8020000000000000,SubjectUserSid=S-1-5-21-143320146-2996204461-2...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},SWAttckd.swtestnet.com,0,2,anomaly,2024-09-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171622,0,5156,4719498,2024-09-26 18:48:50.349300+00:00,Security,12810,Microsoft-Windows-Security-Auditing,NaN,4,6840,NaN,0x8020000000000000,ProcessID=1284|Application=\device\harddiskvol...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},SWAttckd.swtestnet.com,0,1,anomaly,2024-09-26
171624,0,5156,4719500,2024-09-26 18:48:50.512531+00:00,Security,12810,Microsoft-Windows-Security-Auditing,NaN,4,6840,NaN,0x8020000000000000,ProcessID=6912|Application=\device\harddiskvol...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},SWAttckd.swtestnet.com,0,1,anomaly,2024-09-26
171627,0,5156,4719503,2024-09-26 18:48:50.929255+00:00,Security,12810,Microsoft-Windows-Security-Auditing,NaN,4,300,NaN,0x8020000000000000,ProcessID=6912|Application=\device\harddiskvol...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},SWAttckd.swtestnet.com,0,1,anomaly,2024-09-26
171912,0,4688,4719788,2024-09-26 18:55:46.865633+00:00,Security,13312,Microsoft-Windows-Security-Auditing,NaN,4,4972,NaN,0x8020000000000000,SubjectUserSid=S-1-5-21-143320146-2996204461-2...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},SWAttckd.swtestnet.com,0,2,anomaly,2024-09-26


### パース(Drain)
- ここからのsecurity.csvは10日間を抽出したものを想定！！！！！！！！！！！！！！！！

In [41]:
project_name = 'T1105'
ver_number = "2"
input_dir = INTERIM_DIR/project_name
output_dir = INTERIM_DIR/project_name

preprocess.parse_log(input_dir=input_dir, output_dir=output_dir, logfile_name=f'security{ver_number}', parser_type="drain")

Parsing file: ../data/interim/T1105/buffer.csv
Total size after reading CSV: 177827
Parsing done. [Time taken: 0:00:20.552472]


### モデル前データ作成

In [ ]:
ver_number = "2"
input_dir = INTERIM_DIR/project_name
output_dir = PROCESSED_DIR/project_name/f"ver_{ver_number}"
output_dir.mkdir(exist_ok=True)


preprocess.prepare_model_data(
    logdata_filepath = Path(input_dir/"security2_clean_structured.csv"),
    output_dir = output_dir,
    ano_df = df,
)

/home/siwamura/lad2/src/preprocess.py:14: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  


there are 10396 instances (sliding windows) in this dataset

training size 4137
test normal size 6206
test abnormal size 53
training size 6205
test normal size 4138
test abnormal size 53
training size 8274
test normal size 2069
test abnormal size 53
training size 10343


### training

In [5]:
# train
main.main_cli([
    "train",
    "bert/test3",           # conf/bert_config_name.yaml
    #"default.device_id=0",        # ここに好きな key=value を並べる
    #"default.epochs=10",
])

setup
Building Vocab


100%|██████████| 10343/10343 [00:00<00:00, 65955.78it/s]


VOCAB SIZE: 55
before filtering short session
train size  7447
valid size  827


100%|██████████| 8274/8274 [00:02<00:00, 3951.56it/s]


Num of train seqs 7412
Num of valid seqs 827


...

epoch 108 || TRAIN_Loss:1.0165 ||VAL_Loss:1.0076
Best epoch = 90


231it [00:17, 12.90it/s]
25it [00:00, 37.53it/s]


epoch 109 || TRAIN_Loss:1.0159 ||VAL_Loss:1.0081
Best epoch = 90


231it [00:17, 12.89it/s]
25it [00:00, 37.43it/s]


epoch 110 || TRAIN_Loss:1.0168 ||VAL_Loss:1.0149
Early stopping at epoch 110 (best = 90)


In [7]:
# テスト
main.main_cli([
    "test",  # run_mode
    "outputs/logbert/bert/T1105/ver_1/ratio_0.8/seq_len_256/r_seed_31/weights/ValTotalbest.pth", # 重みファイルまでの相対パス
    "32",   # eval_batchsize
    "cuda:0",     # gpu
])

outputs/logbert/bert/T1105/ver_1/ratio_0.8/seq_len_256/r_seed_31
cuda:0 cuda:0
Building Vocab


100%|██████████| 34455/34455 [00:00<00:00, 78005.94it/s]


VOCAB SIZE: 60


100%|██████████| 53/53 [00:00<00:00, 70.81it/s]

Saving test normal results
Saving test abnormal results
[seq_th, FP, TP, TN, FN, Precision, Recall, F1, TPR, FPR]
threshold=0.000['18.0000', '8.0000', '2030.0000', '45.0000', '30.7692', '15.0943', '20.2532', '15.0943', '0.8789']
threshold=0.100['8.0000', '0.0000', '2040.0000', '53.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.3906']
threshold=0.200['0.0000', '0.0000', '2048.0000', '53.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000']
threshold=0.300['0.0000', '0.0000', '2048.0000', '53.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000']
threshold=0.400['0.0000', '0.0000', '2048.0000', '53.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000']
threshold=0.500['0.0000', '0.0000', '2048.0000', '53.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000']
threshold=0.600['0.0000', '0.0000', '2048.0000', '53.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000']
threshold=0.700['0.0000', '0.0000', '2048.0000', '53.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000']
t

## 1-2 VSCode1

### 準備

In [9]:
parent_dir = "ScenarioData"
project_name = 'VSCode1'

### evtx → csv

In [10]:
# test
input_dir = RAW_DIR/parent_dir/project_name
output_dir = INTERIM_DIR/project_name

preprocess.evtx_to_csv_without_eventdata_columns_samplingver(
    evtx_filepath = input_dir/"20240706063537_7EA74D56-6663-313B-2CC1-A7843FCD1AE6/Evtx/Security.evtx",
    output_dir = output_dir,
    output_filename = "security2_test",
)

Writing to CSV: 100%|██████████| 50/50 [00:00<00:00, 68556.78it/s]


In [11]:
# 実行
preprocess.evtx_to_csv_without_eventdata_columns(
    evtx_filepath = input_dir/"20240706063537_7EA74D56-6663-313B-2CC1-A7843FCD1AE6/Evtx/Security.evtx",
    output_dir = output_dir,
    output_filename = "security2",
)

Writing to CSV: 100%|██████████| 93912/93912 [00:01<00:00, 67158.05it/s]


### パース(Drain)

In [ ]:
input_dir = INTERIM_DIR/project_name
output_dir = INTERIM_DIR/project_name

df = pd.read_csv(input_dir/"security.csv")
df = df.dropna(subset=["Content"]) # Contentカラムに欠損値をもつ行を削除
df.to_csv(input_dir / "security_clean.csv", index=False)

In [ ]:
preprocess.parse_log(input_dir=input_dir, output_dir=output_dir, logfile_name='security_clean', parser_type="drain")

## 1-3 WEB01

### 準備

In [20]:
parent_dir = "ScenarioData"
project_name = 'WEB1'
ver_number = "2"

### evtx → csv

In [13]:
# test
input_dir = RAW_DIR/parent_dir/project_name
output_dir = INTERIM_DIR/project_name

preprocess.evtx_to_csv_without_eventdata_columns_samplingver(
    evtx_filepath = input_dir/"20240317044842_7EA74D56-6663-313B-2CC1-A7843FCD1AE6/Evtx/Security.evtx",
    output_dir = output_dir,
    output_filename = "security2_test",
)

Writing to CSV: 100%|██████████| 50/50 [00:00<00:00, 61230.72it/s]


In [14]:
# 実行
preprocess.evtx_to_csv_without_eventdata_columns(
    evtx_filepath = input_dir/"20240317044842_7EA74D56-6663-313B-2CC1-A7843FCD1AE6/Evtx/Security.evtx",
    output_dir = output_dir,
    output_filename = "security2",
)

Writing to CSV: 100%|██████████| 215806/215806 [00:02<00:00, 72085.06it/s]


### 余計な区間を削除（11/22~11/23）

In [21]:
project_name = 'WEB1'
ver_number = "2"
data = pd.read_csv(INTERIM_DIR/project_name/f"security{ver_number}.csv")
data["TimeCreated_SystemTime"] = pd.to_datetime(
    data["TimeCreated_SystemTime"], 
    errors="coerce",      # 変換できないものは NaT にする
)

data["date"] = data["TimeCreated_SystemTime"].dt.date

filtered = data[
    (data["date"] <= pd.to_datetime("2024-11-01").date()) &
    (data["date"] >= pd.to_datetime("2023-12-01").date())
]

# データ上書き注意！！！！！！！！！！！！！！
filtered.to_csv(INTERIM_DIR/project_name/f"security{ver_number}.csv", index=False)

### アノテーション

In [ ]:
project_name = 'WEB1'
ver_number = "2"
ano = pd.read_excel(RAW_DIR/parent_dir/"_anotation"/project_name/"web1_labeled_incident_data.xlsx")
df = pd.DataFrame()
df["Channel"] = ano["Channel"].values
df["EventRecordID"] = ano["RecordID"].values

labeled = preprocess.anotate_csv(
    csv_filepath = INTERIM_DIR/project_name/"security2.csv",
    ano_df = df,
    output_dir = INTERIM_DIR/project_name,
)

#### 異常期間の同定
- **期間**：3/14 15:00 ~ 3/17 16:48 (21162件)

In [27]:
labeled[labeled["Label"] == "anomaly"]

,TimeCreated_SystemTime,Security_UserID,Correlation_ActivityID,Channel,Version,Content,Correlation_RelatedActivityID,Provider_Guid,Keywords,Opcode,EventID,Provider_Name,Computer,Task,EventRecordID,Level,Execution_ProcessID,Execution_ThreadID,Label
144356,2024-03-14 15:00:01.005144+00:00,NaN,NaN,Security,2,SubjectUserSid=S-1-5-18|SubjectUserName=SWATTC...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},0x8020000000000000,0,4688,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,13312,1511462,0,4,4488,anomaly
144358,2024-03-14 15:00:49.232059+00:00,NaN,NaN,Security,1,ProcessID=2572|Application=\device\harddiskvol...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},0x8020000000000000,0,5156,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,12810,1511464,0,4,4488,anomaly
144361,2024-03-14 15:01:49.291647+00:00,NaN,NaN,Security,1,ProcessID=1264|Application=\device\harddiskvol...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},0x8020000000000000,0,5156,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,12810,1511467,0,4,4488,anomaly
144363,2024-03-14 15:01:49.317078+00:00,NaN,NaN,Security,1,ProcessID=2572|Application=\device\harddiskvol...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},0x8020000000000000,0,5156,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,12810,1511469,0,4,4488,anomaly
144365,2024-03-14 15:02:49.371119+00:00,NaN,NaN,Security,1,ProcessID=2572|Application=\device\harddiskvol...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},0x8020000000000000,0,5156,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,12810,1511471,0,4,4488,anomaly
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194059,2024-03-17 16:48:45.044725+00:00,NaN,NaN,Security,1,ProcessID=4|Application=System|Direction=%%145...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},0x8020000000000000,0,5156,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,12810,1561165,0,4,2028,anomaly
194060,2024-03-17 16:48:45.211576+00:00,NaN,NaN,Security,2,SubjectUserSid=S-1-5-18|SubjectUserName=SWATTC...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},0x8020000000000000,0,4688,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,13312,1561166,0,4,2656,anomaly
194061,2024-03-17 16:48:45.240389+00:00,NaN,NaN,Security,2,SubjectUserSid=S-1-5-18|SubjectUserName=SWATTC...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},0x8020000000000000,0,4688,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,13312,1561167,0,4,7440,anomaly
194071,2024-03-17 16:48:46.617451+00:00,NaN,NaN,Security,2,SubjectUserSid=S-1-5-18|SubjectUserName=SWATTC...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},0x8020000000000000,0,4688,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,13312,1561177,0,4,5188,anomaly


### 

### パース(Drain)

In [24]:
input_dir = INTERIM_DIR/project_name
output_dir = INTERIM_DIR/project_name
preprocess.parse_log(input_dir=input_dir, output_dir=output_dir, logfile_name='security2', parser_type="drain")

Parsing file: ../data/interim/WEB1/buffer.csv
Total size after reading CSV: 194072
Parsing done. [Time taken: 0:00:22.991458]


## 1-4 WEB02

### 準備

In [28]:
parent_dir = "ScenarioData"
project_name = 'WEB2'
ver_number = "2"

### evtx → csv

In [ ]:
# test
input_dir = RAW_DIR/parent_dir/project_name
output_dir = INTERIM_DIR/project_name

preprocess.evtx_to_csv_without_eventdata_columns_samplingver(
    evtx_filepath = input_dir/"20240317051629_7EA74D56-6663-313B-2CC1-A7843FCD1AE6/Evtx/Security.evtx",
    output_dir = output_dir,
    output_filename = f"security{ver_number}_test",
)

Writing to CSV: 100%|██████████| 50/50 [00:00<00:00, 46386.91it/s]


In [32]:
# 実行
preprocess.evtx_to_csv_without_eventdata_columns(
    evtx_filepath = input_dir/"20240317051629_7EA74D56-6663-313B-2CC1-A7843FCD1AE6/Evtx/Security.evtx",
    output_dir = output_dir,
    output_filename = f"security{ver_number}",
)

Writing to CSV: 100%|██████████| 215808/215808 [00:03<00:00, 64105.31it/s]


### アノテーション

In [29]:
project_name = 'WEB2'
ver_number = "2"
ano = pd.read_excel(RAW_DIR/parent_dir/"_anotation"/project_name/"web2_labeled_incident_data.xlsx")
df = pd.DataFrame()
df["Channel"] = ano["Channel"].values
df["EventRecordID"] = ano["RecordID"].values

labeled = preprocess.anotate_csv(
    csv_filepath = INTERIM_DIR/project_name/"security2.csv",
    ano_df = df,
    output_dir = INTERIM_DIR/project_name,
)

#### 異常期間の同定
- **期間**：3/14 15:00 ~ 3/17 17:17 (21683件)

In [39]:
labeled[labeled["Label"] == "anomaly"]

,Security_UserID,Level,Keywords,Channel,Content,EventID,Correlation_ActivityID,TimeCreated_SystemTime,Correlation_RelatedActivityID,Task,Execution_ProcessID,Execution_ThreadID,Provider_Name,Computer,Version,Opcode,EventRecordID,Provider_Guid,Label
144356,NaN,0,0x8020000000000000,Security,SubjectUserSid=S-1-5-18|SubjectUserName=SWATTC...,4688,NaN,2024-03-14 15:00:01.005144+00:00,NaN,13312,4,4488,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,2,0,1511462,{54849625-5478-4994-a5ba-3e3b0328c30d},anomaly
144358,NaN,0,0x8020000000000000,Security,ProcessID=2572|Application=\device\harddiskvol...,5156,NaN,2024-03-14 15:00:49.232059+00:00,NaN,12810,4,4488,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,1,0,1511464,{54849625-5478-4994-a5ba-3e3b0328c30d},anomaly
144361,NaN,0,0x8020000000000000,Security,ProcessID=1264|Application=\device\harddiskvol...,5156,NaN,2024-03-14 15:01:49.291647+00:00,NaN,12810,4,4488,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,1,0,1511467,{54849625-5478-4994-a5ba-3e3b0328c30d},anomaly
144363,NaN,0,0x8020000000000000,Security,ProcessID=2572|Application=\device\harddiskvol...,5156,NaN,2024-03-14 15:01:49.317078+00:00,NaN,12810,4,4488,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,1,0,1511469,{54849625-5478-4994-a5ba-3e3b0328c30d},anomaly
144365,NaN,0,0x8020000000000000,Security,ProcessID=2572|Application=\device\harddiskvol...,5156,NaN,2024-03-14 15:02:49.371119+00:00,NaN,12810,4,4488,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,1,0,1511471,{54849625-5478-4994-a5ba-3e3b0328c30d},anomaly
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195225,NaN,0,0x8020000000000000,Security,ProcessID=1264|Application=\device\harddiskvol...,5156,NaN,2024-03-17 17:16:04.655096+00:00,NaN,12810,4,6388,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,1,0,1562331,{54849625-5478-4994-a5ba-3e3b0328c30d},anomaly
195228,NaN,0,0x8020000000000000,Security,ProcessID=2572|Application=\device\harddiskvol...,5156,NaN,2024-03-17 17:16:28.634140+00:00,NaN,12810,4,3960,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,1,0,1562334,{54849625-5478-4994-a5ba-3e3b0328c30d},anomaly
195229,NaN,0,0x8020000000000000,Security,SubjectUserSid=S-1-5-18|SubjectUserName=SWATTC...,4688,NaN,2024-03-17 17:16:28.766081+00:00,NaN,13312,4,3960,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,2,0,1562335,{54849625-5478-4994-a5ba-3e3b0328c30d},anomaly
195230,NaN,0,0x8020000000000000,Security,SubjectUserSid=S-1-5-18|SubjectUserName=SWATTC...,4688,NaN,2024-03-17 17:16:29.024967+00:00,NaN,13312,4,8988,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,2,0,1562336,{54849625-5478-4994-a5ba-3e3b0328c30d},anomaly


### パース(Drain)

In [30]:
project_name = 'WEB2'
ver_number = "2"
input_dir = INTERIM_DIR/project_name
output_dir = INTERIM_DIR/project_name
preprocess.parse_log(input_dir=input_dir, output_dir=output_dir, logfile_name='security2', parser_type="drain")

Parsing file: ../data/interim/WEB2/buffer.csv
Total size after reading CSV: 215806
Parsing done. [Time taken: 0:00:25.667182]


## 1-5 WEB03

### 準備

In [33]:
parent_dir = "ScenarioData"
project_name = 'WEB3'
ver_number = "2"

### evtx → csv

In [34]:
# test
input_dir = RAW_DIR/parent_dir/project_name
output_dir = INTERIM_DIR/project_name

preprocess.evtx_to_csv_without_eventdata_columns_samplingver(
    evtx_filepath = input_dir/"20240317054607_7EA74D56-6663-313B-2CC1-A7843FCD1AE6/Evtx/Security.evtx",
    output_dir = output_dir,
    output_filename = f"security{ver_number}_test",
)

Writing to CSV: 100%|██████████| 50/50 [00:00<00:00, 46172.44it/s]


In [35]:
# 実行
preprocess.evtx_to_csv_without_eventdata_columns(
    evtx_filepath = input_dir/"20240317054607_7EA74D56-6663-313B-2CC1-A7843FCD1AE6/Evtx/Security.evtx",
    output_dir = output_dir,
    output_filename = f"security{ver_number}",
)

Writing to CSV: 100%|██████████| 215773/215773 [00:03<00:00, 62537.91it/s]


### アノテーション

### パース(Drain)

In [17]:
input_dir = INTERIM_DIR/project_name
output_dir = INTERIM_DIR/project_name

df = pd.read_csv(input_dir/"security.csv")
df = df.dropna(subset=["Content"]) # Contentカラムに欠損値をもつ行を削除
df.to_csv(input_dir / "security_clean.csv", index=False)

In [18]:
preprocess.parse_log(input_dir=input_dir, output_dir=output_dir, logfile_name='security_clean', parser_type="drain")

Parsing file: ../data/interim/WEB3/security_clean.csv
Initial DataFrame:
    Task  Level                Computer  EventID  \
0  12804      0  SWAttckd.swtestnet.com     4702   
1  12804      0  SWAttckd.swtestnet.com     4702   
2  12804      0  SWAttckd.swtestnet.com     4702   
3  12804      0  SWAttckd.swtestnet.com     4702   
4  12810      0  SWAttckd.swtestnet.com     5158   

                   Correlation_ActivityID  \
0  {326660a5-67c0-0001-6861-6632c067da01}   
1  {326660a5-67c0-0001-6861-6632c067da01}   
2  {326660a5-67c0-0001-6861-6632c067da01}   
3  {326660a5-67c0-0001-6861-6632c067da01}   
4                                     NaN   

                                             Content  EventRecordID  \
0  SubjectUserSid=S-1-5-18;SubjectUserName=SWATTC...        1367106   
1  SubjectUserSid=S-1-5-18;SubjectUserName=SWATTC...        1367107   
2  SubjectUserSid=S-1-5-18;SubjectUserName=SWATTC...        1367108   
3  SubjectUserSid=S-1-5-18;SubjectUserName=SWATTC...       

## 1-6 総合データ（Integrated）

### csv ⇒ 統合csv

In [15]:
input_dir = output_dir = INTERIM_DIR/"Integrated"

project_list = ["T1105", "WEB1", "WEB2"]

# T1105 だけフルデータが別ディレクトリ下なので、別扱い
df = pd.read_csv(NO_MEANING_DIR/"T1105/security2_complement.csv")
df["project"] = "T1105_C"

for project in project_list:
    dir = Path(INTERIM_DIR / project / "security2.csv")
    temp_df = pd.read_csv(dir)
    temp_df["project"] = str(project)    
    df = pd.concat([df, temp_df], ignore_index=True)

df.to_csv(output_dir/"security_integrated.csv", index=False)

### パース(Drain)

In [16]:
input_dir = output_dir = INTERIM_DIR/"Integrated"
preprocess.parse_log(input_dir=input_dir, output_dir=output_dir, logfile_name='security_integrated', parser_type="drain", integrated=True)

Parsing file: ../data/interim/Integrated/buffer.csv
Total size after reading CSV: 1416987
Parsing done. [Time taken: 0:03:10.633761]


### モデル前データ作成

In [17]:
preprocess.prepare_integrated_model_data(
    logdata_filepath = INTERIM_DIR/"Integrated"/"security_integrated_structured.csv", 
    output_dir = PROCESSED_DIR/"Integrated",
    project_list = ["T1105_C","T1105", "WEB1", "WEB2"]
)

/home/siwamura/My_lad/src/preprocess.py:739: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  data["timestamp"] = data["datetime"].view("int64") // 10**9


there are 42619 instances (sliding windows) in this dataset

there are 10396 instances (sliding windows) in this dataset

there are 16485 instances (sliding windows) in this dataset

there are 16512 instances (sliding windows) in this dataset

there are 42619 instances (sliding windows) in this dataset

there are 10396 instances (sliding windows) in this dataset

there are 16485 instances (sliding windows) in this dataset

there are 16512 instances (sliding windows) in this dataset

there are 42619 instances (sliding windows) in this dataset

there are 10396 instances (sliding windows) in this dataset

there are 16485 instances (sliding windows) in this dataset

there are 16512 instances (sliding windows) in this dataset

vocab size 34455


### 実行

In [ ]:
# train
main.main_cli([
    "train",
    "bert/test",           # conf/bert_config_name.yaml
    #"default.device_id=0",        # ここに好きな key=value を並べる
    #"default.epochs=10",
])

setup
Building Vocab


100%|██████████| 34455/34455 [00:00<00:00, 72592.75it/s]


VOCAB SIZE: 60
before filtering short session
train size  63163
valid size  7018


100%|██████████| 70181/70181 [00:18<00:00, 3767.41it/s]


Num of train seqs 63127
Num of valid seqs 7018


...

epoch 2 || TRAIN_Loss:1.1468 ||VAL_Loss:1.0919
Best epoch = 2


1972it [01:58, 16.60it/s]
219it [00:03, 71.58it/s]


epoch 3 || TRAIN_Loss:1.1078 ||VAL_Loss:1.0852
Best epoch = 3


1972it [01:59, 16.49it/s]
105it [00:01, 67.78it/s]


RuntimeError: CUDA error: an illegal instruction was encountered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
"""
HyperSphereLossなしでテスト
Jupyter Notebookで実行してください
"""

import sys
sys.path.insert(0, '/home/siwamura/My_lad/src')

from main import main_cli

print("=" * 80)
print("🚀 HyperSphereLoss無効化テスト")
print("=" * 80)
print("HyperSphereLossを無効化してトレーニングします")
print("これでNaNが消えれば、HyperSphereLossが原因です")
print("=" * 80)

try:
    main_cli([
        "train",
        "bert/test",
        "dataset.sample.seq_len=128",
        "default.epochs=2",
        "loss.hypersphere.bias=0.0",  # HyperSphereLossを無効化
        "loss.mask.bias=1.0",  # MaskLossのみ使用
    ])
    
    print("\n" + "=" * 80)
    print("✅ トレーニング完了（HyperSphereLossなし）")
    print("=" * 80)
    print("\n結論:")
    print("  HyperSphereLossを無効化すれば正常に動作する")
    print("  → HyperSphereLossの実装に問題がある")

except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()


## 補足

### "Channel="を削ぐ
- 一応過程として残す

In [14]:
def remove_leading_channel_security(s):
    # NaN はそのまま返す
    if pd.isna(s):
        return s

    text = str(s)
    parts = text.split(";")

    # 先頭がちょうど "Channel=Security" なら、それを落とす
    if parts and parts[0] == "Channel=Security":
        parts = parts[1:]

    # 空文字を除いて再結合
    parts = [p for p in parts if p != ""]
    return ";".join(parts)

In [30]:
project_name = "WEB1"
input_dir = INTERIM_DIR/project_name

df = pd.read_csv(input_dir/"security.csv")
# Content カラムに適用
df["Content"] = df["Content"].apply(remove_leading_channel_security)
# ""をNaNに変換
df.loc[df["Content"] == "", "Content"] = np.nan

In [31]:
# 保存
df.to_csv(input_dir/"security_.csv", index=False)

### 余計なデータを消す
scenario：削除対象 [正常期間]
- VSCode1：(2024)5/22 [(2024)6/27~7/6]
- WEB1：(2023)11/22 ~ 11/23 [(2024)3/6~3/17]
- WEB2：(2023)11/22 ~ 11/23 [(2024)3/6~3/17]


In [11]:
preprocess.delete_unwanted_logs(
    input_filepath=Path(INTERIM_DIR /"VSCode1"/ "security2.csv"),
    start_date="2024-06-01",
    end_date="2024-07-30",
)

,TimeCreated_SystemTime,Security_UserID,Correlation_ActivityID,Channel,Version,Content,Correlation_RelatedActivityID,Provider_Guid,Keywords,Opcode,EventID,Provider_Name,Computer,Task,EventRecordID,Level,Execution_ProcessID,Execution_ThreadID,date
7458,2024-06-27 05:12:47.146389+00:00,NaN,NaN,Security,1,SubjectUserSid=S-1-5-18|SubjectUserName=SWATTC...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},0x8020000000000000,0,4616,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,12288,2543604,0,4,6596,2024-06-27
7459,2024-06-27 05:12:48.635813+00:00,NaN,NaN,Security,2,SubjectUserSid=S-1-5-18|SubjectUserName=SWATTC...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},0x8020000000000000,0,4688,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,13312,2543605,0,4,6596,2024-06-27
7460,2024-06-27 05:12:50.183855+00:00,NaN,NaN,Security,2,SubjectUserSid=S-1-5-18|SubjectUserName=SWATTC...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},0x8020000000000000,0,4688,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,13312,2543606,0,4,2332,2024-06-27
7461,2024-06-27 05:12:50.618172+00:00,NaN,NaN,Security,2,SubjectUserSid=S-1-5-18|SubjectUserName=SWATTC...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},0x8020000000000000,0,4688,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,13312,2543607,0,4,2648,2024-06-27
7462,2024-06-27 05:12:50.619019+00:00,NaN,NaN,Security,2,SubjectUserSid=S-1-5-18|SubjectUserName=SWATTC...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},0x8020000000000000,0,4688,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,13312,2543608,0,4,2988,2024-06-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93907,2024-07-06 18:35:40.437977+00:00,NaN,NaN,Security,0,ProcessId=676|Application=\device\harddiskvolu...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},0x8020000000000000,0,5158,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,12810,2630053,0,4,5552,2024-07-06
93908,2024-07-06 18:35:40.438011+00:00,NaN,NaN,Security,1,ProcessID=676|Application=\device\harddiskvolu...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},0x8020000000000000,0,5156,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,12810,2630054,0,4,5552,2024-07-06
93909,2024-07-06 18:35:40.498657+00:00,NaN,NaN,Security,2,SubjectUserSid=S-1-5-18|SubjectUserName=SWATTC...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},0x8020000000000000,0,4688,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,13312,2630055,0,4,6436,2024-07-06
93910,2024-07-06 18:35:40.527555+00:00,NaN,NaN,Security,2,SubjectUserSid=S-1-5-18|SubjectUserName=SWATTC...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},0x8020000000000000,0,4688,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,13312,2630056,0,4,6436,2024-07-06


In [12]:
preprocess.delete_unwanted_logs(
    input_filepath=Path(INTERIM_DIR /"WEB1"/ "security2.csv"),
    start_date="2024-03-01",
    end_date="2024-03-30",
)

,TimeCreated_SystemTime,Security_UserID,Correlation_ActivityID,Channel,Version,Content,Correlation_RelatedActivityID,Provider_Guid,Keywords,Opcode,EventID,Provider_Name,Computer,Task,EventRecordID,Level,Execution_ProcessID,Execution_ThreadID,date,Label
0,2024-03-06 05:48:33.170567+00:00,NaN,{326660a5-67c0-0001-6861-6632c067da01},Security,1,SubjectUserSid=S-1-5-18|SubjectUserName=SWATTC...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},0x8020000000000000,0,4702,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,12804,1367106,0,692,6000,2024-03-06,-
1,2024-03-06 05:48:33.174620+00:00,NaN,{326660a5-67c0-0001-6861-6632c067da01},Security,1,SubjectUserSid=S-1-5-18|SubjectUserName=SWATTC...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},0x8020000000000000,0,4702,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,12804,1367107,0,692,6000,2024-03-06,-
2,2024-03-06 05:48:33.180763+00:00,NaN,{326660a5-67c0-0001-6861-6632c067da01},Security,1,SubjectUserSid=S-1-5-18|SubjectUserName=SWATTC...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},0x8020000000000000,0,4702,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,12804,1367108,0,692,6000,2024-03-06,-
3,2024-03-06 05:48:33.184887+00:00,NaN,{326660a5-67c0-0001-6861-6632c067da01},Security,1,SubjectUserSid=S-1-5-18|SubjectUserName=SWATTC...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},0x8020000000000000,0,4702,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,12804,1367109,0,692,6000,2024-03-06,-
4,2024-03-06 05:48:38.115097+00:00,NaN,NaN,Security,0,ProcessId=8596|Application=\device\harddiskvol...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},0x8020000000000000,0,5158,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,12810,1367110,0,4,5136,2024-03-06,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194069,2024-03-17 16:48:46.498323+00:00,NaN,{81f03b91-7558-0001-6c3c-f0815875da01},Security,0,TargetUserName=WDAGUtilityAccount|TargetDomain...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},0x8020000000000000,0,4798,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,13824,1561175,0,680,8556,2024-03-17,-
194070,2024-03-17 16:48:46.498625+00:00,NaN,{81f03b91-7558-0001-6c3c-f0815875da01},Security,0,TargetUserName=一般|TargetDomainName=SWATTCKD|Ta...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},0x8020000000000000,0,4798,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,13824,1561176,0,680,8556,2024-03-17,-
194071,2024-03-17 16:48:46.617451+00:00,NaN,NaN,Security,2,SubjectUserSid=S-1-5-18|SubjectUserName=SWATTC...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},0x8020000000000000,0,4688,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,13312,1561177,0,4,5188,2024-03-17,anomaly
194072,2024-03-17 16:48:46.646482+00:00,NaN,NaN,Security,2,SubjectUserSid=S-1-5-18|SubjectUserName=SWATTC...,NaN,{54849625-5478-4994-a5ba-3e3b0328c30d},0x8020000000000000,0,4688,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,13312,1561178,0,4,2656,2024-03-17,anomaly


In [14]:
preprocess.delete_unwanted_logs(
    input_filepath=Path(INTERIM_DIR /"WEB2"/ "security2.csv"),
    start_date="2024-03-01",
    end_date="2024-03-30",
)

,Security_UserID,Level,Keywords,Channel,Content,EventID,Correlation_ActivityID,TimeCreated_SystemTime,Correlation_RelatedActivityID,Task,Execution_ProcessID,Execution_ThreadID,Provider_Name,Computer,Version,Opcode,EventRecordID,Provider_Guid,Label,date
0,NaN,0,0x8020000000000000,Security,SubjectUserSid=S-1-5-18|SubjectUserName=SWATTC...,4702,{326660a5-67c0-0001-6861-6632c067da01},2024-03-06 05:48:33.170567+00:00,NaN,12804,692,6000,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,1,0,1367106,{54849625-5478-4994-a5ba-3e3b0328c30d},-,2024-03-06
1,NaN,0,0x8020000000000000,Security,SubjectUserSid=S-1-5-18|SubjectUserName=SWATTC...,4702,{326660a5-67c0-0001-6861-6632c067da01},2024-03-06 05:48:33.174620+00:00,NaN,12804,692,6000,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,1,0,1367107,{54849625-5478-4994-a5ba-3e3b0328c30d},-,2024-03-06
2,NaN,0,0x8020000000000000,Security,SubjectUserSid=S-1-5-18|SubjectUserName=SWATTC...,4702,{326660a5-67c0-0001-6861-6632c067da01},2024-03-06 05:48:33.180763+00:00,NaN,12804,692,6000,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,1,0,1367108,{54849625-5478-4994-a5ba-3e3b0328c30d},-,2024-03-06
3,NaN,0,0x8020000000000000,Security,SubjectUserSid=S-1-5-18|SubjectUserName=SWATTC...,4702,{326660a5-67c0-0001-6861-6632c067da01},2024-03-06 05:48:33.184887+00:00,NaN,12804,692,6000,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,1,0,1367109,{54849625-5478-4994-a5ba-3e3b0328c30d},-,2024-03-06
4,NaN,0,0x8020000000000000,Security,ProcessId=8596|Application=\device\harddiskvol...,5158,NaN,2024-03-06 05:48:38.115097+00:00,NaN,12810,4,5136,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,0,0,1367110,{54849625-5478-4994-a5ba-3e3b0328c30d},-,2024-03-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195229,NaN,0,0x8020000000000000,Security,SubjectUserSid=S-1-5-18|SubjectUserName=SWATTC...,4688,NaN,2024-03-17 17:16:28.766081+00:00,NaN,13312,4,3960,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,2,0,1562335,{54849625-5478-4994-a5ba-3e3b0328c30d},anomaly,2024-03-17
195230,NaN,0,0x8020000000000000,Security,SubjectUserSid=S-1-5-18|SubjectUserName=SWATTC...,4688,NaN,2024-03-17 17:16:29.024967+00:00,NaN,13312,4,8988,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,2,0,1562336,{54849625-5478-4994-a5ba-3e3b0328c30d},anomaly,2024-03-17
195231,NaN,0,0x8020000000000000,Security,SubjectUserSid=S-1-5-18|SubjectUserName=SWATTC...,4688,NaN,2024-03-17 17:16:29.153864+00:00,NaN,13312,4,3960,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,2,0,1562337,{54849625-5478-4994-a5ba-3e3b0328c30d},anomaly,2024-03-17
195232,NaN,0,0x8020000000000000,Security,SubjectUserSid=S-1-5-18|SubjectUserName=SWATTC...,4673,NaN,2024-03-17 17:16:29.450386+00:00,NaN,13056,4,8988,Microsoft-Windows-Security-Auditing,SWAttckd.swtestnet.com,0,0,1562338,{54849625-5478-4994-a5ba-3e3b0328c30d},-,2024-03-17


### T1105のフルバージョンのstructuredを作成

In [8]:
preprocess.parse_log(
    input_dir=NO_MEANING_DIR/"T1105", 
    output_dir=NO_MEANING_DIR/"T1105", 
    logfile_name='security2', 
    parser_type="drain", 
)

Parsing file: ../data/no_meaning/T1105/buffer.csv
Total size after reading CSV: 1027687
Parsing done. [Time taken: 0:02:07.250112]


### HDFS